# Setup

In [1]:
# mount drive 
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
cd "/content/gdrive/My Drive/Github/SubjectIndexing"

/content/gdrive/My Drive/Github/SubjectIndexing


In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

# Import data

In [4]:
# import document embeddings
with open('./data/books_B_embeddings.pkl', 'rb') as handle:
    embeddings_1 = pickle.load(handle)
with open('./data/books_B_embeddings_p2.pkl', 'rb') as handle:
    embeddings_2 = pickle.load(handle)
with open('./data/books_B_embeddings_p3.pkl', 'rb') as handle:
    embeddings_3 = pickle.load(handle)
# embeddings = np.mean([embeddings_1, embeddings_2, embeddings_3], axis=0)
embeddings = embeddings_1

In [12]:
# prepare metadata
with open("./data/books_B.json", "r") as f:
    books_B = json.load(f) 
books_B = {int(k):v for k,v in books_B.items()}  # convert str keys to int
books_B = dict(sorted(books_B.items()))  # sort books
metadata = pd.read_json('./data/metadata_nosuffix.json')
metadata_B = metadata[metadata['document'].isin(list(books_B.keys()))]  # filter books in class B
metadata_B = metadata_B.sort_values(by=['document'])  # sort metadata

In [14]:
metadata_B.head()

,document,title,creator,subjects,subjects_new
54384,59,Discourse on the Method of Rightly Conducting ...,"[Descartes, René]","[Science -- Methodology, B, Methodology]","[Science, B, Methodology]"
63619,919,Ethics — Part 1,"[Spinoza, Benedictus de]","[B, Ethics]","[B, Ethics]"
63631,920,Ethics — Part 2,"[Spinoza, Benedictus de]","[Ethics, B]","[Ethics, B]"
63941,948,Ethics — Part 3,"[Spinoza, Benedictus de]","[B, Ethics]","[B, Ethics]"
64197,971,Ethics — Part 4,"[Spinoza, Benedictus de]","[Ethics, B]","[Ethics, B]"


In [15]:
# count subject frequencies
counter = {}
for row in metadata.subjects_new:
    for sub in row:
        if sub not in counter:
            counter[sub] = 1
        else:
            counter[sub] += 1
counter_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)